<a href="https://colab.research.google.com/github/mariarodrius/AI-with-Copilot/blob/main/Deploy_Translate_Summarize_Generate%20Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer, MarianMTModel, MarianTokenizer
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import torchaudio

@st.cache_resource
def load_model_and_tokenizer():
    summarization_tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")
    summarization_model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed")
    translation_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
    translation_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
    return summarization_model, summarization_tokenizer, translation_model, translation_tokenizer

@st.cache_resource
def load_tts_model():
    config = XttsConfig()
    config.load_json("/path/to/xtts/config.json")
    model = Xtts.init_from_config(config)
    model.load_checkpoint(config, checkpoint_dir="/path/to/xtts/", eval=True)
    model.cuda()
    return model, config

summarization_model, summarization_tokenizer, translation_model, translation_tokenizer = load_model_and_tokenizer()
tts_model, tts_config = load_tts_model()

st.title("Text Translation, Summarization, and TTS")

text = st.text_area("Enter the text to process:")

if st.button("Translate, Summarize, and Generate Voice"):
    if text:
        with st.spinner("Translating..."):
            # Translation
            translation_inputs = translation_tokenizer(text, return_tensors="pt", truncation=True)
            translated_ids = translation_model.generate(**translation_inputs)
            translated_text = translation_tokenizer.decode(translated_ids[0], skip_special_tokens=True)

            with st.spinner("Summarizing..."):
                # Summarization
                summarization_inputs = summarization_tokenizer(translated_text, return_tensors='pt', truncation=True)
                summary_ids = summarization_model.generate(summarization_inputs['input_ids'], max_length=150, num_beams=2, early_stopping=True)
                summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                st.subheader("Summary:")
                st.write(summary)

            with st.spinner("Generating Voice..."):
                # Text-to-Speech
                outputs = tts_model.synthesize(
                    summary,
                    tts_config,
                    speaker_wav="/data/TTS-public/_refclips/3.wav",
                    gpt_cond_len=3,
                    language="en",
                )
                # Save the output to a file
                output_path = "output.wav"
                torchaudio.save(output_path, outputs['waveform'].cpu(), outputs['sampling_rate'])

                # Provide a download link for the audio file
                st.audio(output_path)
                st.success("Voice generation complete.")
    else:
        st.error("Please enter some text to process.")

In [ ]:
!streamlit run app.py --server.port 8511 & npx localtunnel --port 8511

## EXternal URL - last 5 digits including (':' + port number) is the localtunnel password requested on the website




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8511
  Network URL: http://172.28.0.12:8511
  External URL: http://34.125.145.153:8511

npx: installed 22 in 3.902s
your url is: https://five-views-notice.loca.lt
tokenizer_config.json: 100% 1.20k/1.20k [00:00<00:00, 7.98MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 1.05k/1.05k [00:00<00:00, 7.54MB/s]
spiece.model: 100% 1.92M/1.92M [00:00<00:00, 4.63MB/s]
tokenizer.json: 100% 3.51M/3.51M [00:00<00:00, 10.2MB/s]
special_tokens_map.json: 100% 775/775 [00:00<00:00, 5.47MB/s]
pytorch_model.bin: 100% 2.31G/2.31G [00:45<00:00, 50.2MB/s]
generation_config.json: 100% 232/232 [00:00<00:00, 1.66MB/s]
Input ids are automatically padded from 743 to 768 